In [ ]:
matplotlib notebook

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import mne

In [ ]:
# getting info without loading data
# reading locally:
raw = mne.io.read_raw_edf('a1d36553_8.edf',preload = False)
# reading from cylon:
# raw = mne.io.read_raw_edf('/data1/edf/a1d36553_8/a1d36553_8.edf',preload = False)

In [ ]:
raw.plot(block=True, lowpass=40)

In [ ]:
# Checking out the file info
info = raw.info
print(info)

In [ ]:
# channel names
info['ch_names']

Now let's convert the times to datetime.

In [ ]:
import datetime

In [ ]:
basetime_posix = info['meas_date']

In [ ]:
# Check out the date
datetime.datetime.fromtimestamp(
        basetime_posix
    ).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
channel1 = raw.get_data(picks=1, start=0, stop=400000, reject_by_annotation=None, return_times=True)

In [ ]:
channel1[1].ravel().shape

In [ ]:
import pandas as pd

datetimes = pd.to_datetime(channel1[1].ravel()+basetime_posix, unit = 's')

In [ ]:
plt.figure()
plt.plot(datetimes, channel1[0].ravel())
plt.plot()
plt.title('The first channel')

Seems there is some crazy stuff happening at the beginning of the session. Let's skip the beginning.

In [ ]:
plt.figure()
plt.plot(datetimes[200000:], channel1[0].ravel()[200000:])
plt.plot()
plt.title('The first channel')